For this assignment you will be implementing your own verision of the perceptron algorithm.  Similar to the Naive Bayes assignment your Percptron class should be built to be compatible with the sklearn framework.  You can use the sklearn library and other python libraries to help with your implementation but you must implement the actual percpetron algorithm using your own code.

In [1]:
# import required modules
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, balanced_accuracy_score # balanced_accuracy_score with adjusted=True is Informedness
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
import numpy as np

In [2]:
#TODO: put your perceptron implemenation here

#initial imports that you may find useful
from sklearn.base import BaseEstimator, ClassifierMixin
import itertools 

class perceptron(BaseEstimator, ClassifierMixin):
    def __init__(self, alpha=0.1, epochs=1):
        self.alpha = alpha
        self.epochs = epochs
                 
    def predict(self, X):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        y_hat = np.zeros(len(X))

        for i, row in enumerate(X):                      # Iterate through both X and y together. i is the counter, row is X[i]
            # print(f"predict.enumerate.row: {row}")
            a = sigmoid(np.sum(np.multiply(weights,row)))
            # print(f"a: {a} from sigmoid(sum({weights} product {row}))")

            if a >= 0.5:
                # print(f"Class: 1")
                y_hat[i] = 1
            else:
                # print(f"Class: -1")
                y_hat[i] = -1
        
        # print(f"y_hat in predict: {y_hat}")
        return np.array(y_hat)
    
    def train_weights(self, X, y, y_predicted, weights, alpha):
        X_rows, X_columns = X.shape
        new_weights = np.zeros((X_columns))
        weights_subtotal = np.zeros((X_columns))
        # print(f"new_weights initial: {new_weights}")

        for i, row in enumerate(zip(y, y_predicted, X)):
            if row[0] != int(row[1]):
                # temp_weights = np.sum(new_weights, np.multiply(self.alpha, np.multiply(row[2], (row[0] - row[1]))), axis=0)
                temp_weights = np.multiply(self.alpha, np.multiply(row[2], (row[0] - row[1]))) 
                print(f"temp_weights[{i}]: {temp_weights}")
                
                weights_subtotal = np.add(weights_subtotal, temp_weights) # This will keep a running subtotal of the weights by parameter
                print(f"weights_subtotal: {weights_subtotal}\n")
        print(f"np.add(weights + weights_subtotal): {weights} + {weights_subtotal}")
        weights = np.add(weights, weights_subtotal)
        print(f"weights: {weights}")
        return weights
    
    def fit(self, X, y):
        
        # STEP 1: Initialize the weights of the parameters
        X_rows, X_columns = X.shape
        # print(f"X shape: {X_rows}, {X_columns}")
        global weights
        weights = np.random.rand(X_columns) # Create an array with the number of parameters as X
        print(f"Initial random weights: {weights}")

        # STEP 2: Apply the weights to the Nth instance
        
        epoch = 1
        while epoch <= self.epochs:
            print(f"\n\nEpoch: {epoch}")
            y_hat = np.zeros(len(X))
            y_hat = perceptron.predict(self, X)
            # print(f"len(y_hat): {len(y_hat)}")

            # for i, row in enumerate(zip(y, y_hat)):                    # Iterate through both X and y together. i is the counter, row is X[i]
            weights = self.train_weights(X, y, y_hat, weights, self.alpha)
            epoch += 1
        
        return self
    
    # def predict(self, X, y):

    #     return np.array(y_predicted) # For a long time I was returning self



Split the diabetes data into and 80/20 train/test split.


In [3]:
#TODO: split the diabetes data into training/test data

df = pd.read_csv('Datasets for Assignment 4/diabetes.csv')
X = df.drop(columns='class')
y = df['class']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

Using the diabetes training data with k-fold cross-validation (you choose the value for k) plot your perceptron's cross-validation accuracy and MCC as a function of the number of training epochs.  Use a learning rate of 0.10.

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
#TODO: k-fold cross validation with a learning rate of 0.10

pipe = make_pipeline(StandardScaler(), perceptron(alpha=0.1, epochs=10))
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
accuracy
# rmse = mean_squared_error(y_valid, y_pred, squared=False)  # about 0.505


Initial random weights: [0.25226535 0.02696781 0.92444963 0.73833353 0.56855863 0.98886396
 0.30959594 0.49508459]


Epoch: 1
temp_weights[2]: [-0.0903     -1.05388266  0.41418614 -0.6464087  -1.81232289 -0.1052836
  0.71278834  0.84006081]
weights_subtotal: [-0.0903     -1.05388266  0.41418614 -0.6464087  -1.81232289 -0.1052836
  0.71278834  0.84006081]

temp_weights[3]: [-0.0903     -2.41157809 -0.01349566 -1.14698564 -5.89974914 -0.59510703
 -5.42780142  0.16975888]
weights_subtotal: [-0.1806     -3.46546076  0.40069049 -1.79339434 -7.71207203 -0.70039064
 -4.71501308  1.00981969]

temp_weights[5]: [ 0.21433846  0.01964395 -0.01349566 -0.58383658 -0.48835221 -1.37380069
  0.08170479  0.25354662]
weights_subtotal: [ 0.03373846 -3.4458168   0.38719483 -2.37723092 -8.20042424 -2.07419133
 -4.63330829  1.2633663 ]

temp_weights[6]: [ 1.12825383  1.34576507 -0.97577972 -0.52126446  0.35579017 -0.62022619
  0.13453969  1.00763629]
weights_subtotal: [ 1.16199229 -2.10005174 -0.58858489 -2.

/var/folders/m0/ggccpqsn2kj_43nbhzzcn79m0000gp/T/ipykernel_20507/4198523322.py:14: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/m0/ggccpqsn2kj_43nbhzzcn79m0000gp/T/ipykernel_20507/4198523322.py:14: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


0.6384364820846905

Choose a normalization/standardization strategy to apply to the diabetes training data. Using k-fold cross validation and plot the cross validation accuracy and MCC as a function of the number of training epochs.  Repeat for learning rates of 0.90, 0.50, 0.10, 0.01, and 0.0001.

In [5]:
#TODO: k-fold cross validation on normalized/standardized data

How does the normalization/standardization affect the accuracy and MCC scores and number of training epochs?

Answer in this text box

On the scaled data, for each learning rate, how many training epochs are needed to acheive maximize the accuracy and MCC scores?

Answer in this text box

Is there a correlation between learning rates and the number of training epochs needed to acheive the optimal results?  If so, what is the correlation?  If not, why not?

Answer in this text box